In [1]:
from pprint import pprint
import json
import re
import string

from datasets import load_from_disk

In [2]:
def normalize_answer(s):
    def remove_(text):
        """ 불필요한 기호 제거 """
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub("《", " ", text)
        text = re.sub("》", " ", text)
        text = re.sub("<", " ", text)
        text = re.sub(">", " ", text)
        text = re.sub("〈", " ", text)
        text = re.sub("〉", " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [3]:
with open("./datas/nbest_predictions_valid.json") as f:
    predictions = json.load(f)
valid_data = load_from_disk("../input/data/train_dataset/validation")

In [4]:
impossible_list = []
for idx, (key, value) in enumerate(predictions.items()):
    predict = normalize_answer(value[0]['text'])
    ground_truth = normalize_answer(valid_data[idx]['answers']['text'][0])

    if predict != ground_truth:
        is_possible = False
        print(f"{idx}번째 답")
        print(f"질문: {valid_data[idx]['question']}")
        print(f"정답: {ground_truth}")
        for i, other in enumerate(value):
            candidate = normalize_answer(other['text'])
            if candidate == ground_truth:
                is_possible = True
            print(f"{i+1}번째 후보군: {candidate}")
            print(f"score:{other['start_logit']} {other['end_logit']} | prob: {other['probability']}")
            if i == 4:
                if not is_possible:
                    impossible_list.append(idx)
                    print('*****후보군에 정답이 없습니다.*****')
                    print(valid_data[idx])
                break
        print("-" * 50)

1번째 답
질문: 스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?
정답: 1871년
1번째 후보군: 1868년
score:1.2931175231933594 0.8539748787879944 | prob: 0.6834068298339844
2번째 후보군: 1871년
score:0.8101982474327087 0.2298230528831482 | prob: 0.22588346898555756
3번째 후보군: 1868
score:1.2931175231933594 -2.164808511734009 | prob: 0.03339168801903725
4번째 후보군: 1871년 6월에 억스브릿지까지 철도가 완공되었다 이 노선은 이후 1871년
score:0.8101982474327087 -1.9707608222961426 | prob: 0.025013992562890053
5번째 후보군: 1871
score:0.8101982474327087 -2.90885591506958 | prob: 0.009789792820811272
--------------------------------------------------
15번째 답
질문: 피어슨이 다시 의회를 해산했던 년도는?
정답: 1965년
1번째 후보군: 1963년
score:5.097230434417725 3.6067845821380615 | prob: 0.9741737246513367
2번째 후보군: 1963
score:5.097230434417725 -0.4671304523944855 | prob: 0.01657133921980858
3번째 후보군: 63년
score:-0.20081466436386108 3.6067845821380615 | prob: 0.004872194491326809
4번째 후보군: 19
score:5.097230434417725 -2.27839732170105 | prob: 0.002708535408601165
5번째 후보군: 1965년
score:1.1375288963317

In [5]:
print(impossible_list)
print(len(impossible_list))

[18, 27, 28, 29, 31, 33, 36, 39, 63, 67, 74, 76, 77, 78, 79, 80, 86, 104, 109, 115, 116, 120, 121, 123, 125, 128, 129, 133, 138, 145, 146, 155, 158, 171, 176, 183, 191, 204, 213, 222, 224, 231, 232, 237, 239]
45
